In [71]:
import os
import numpy as np
import pandas as pd
import codecs 

In [84]:
from lie.lie_util import *

In [72]:
import numpy as np
ntu_action_labels = [6, 7, 8, 9, 22, 23, 24, 38, 80, 93, 99, 100, 102]

kinect_vibe_extract_joints = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 21, 24, 38]


# Raw_offsets give the rough direction of each joint relative to root joint.
# For example, raw offset of root joint is [0,0,0]; raw offset of left hip is [-1, 0, 0].
# It could simply be [1, 0, 0] for all joint except root. However, this may affect the precision of transformation.
humanact12_raw_offsets = np.array([[0,0,0],
                               [1,0,0],
                               [-1,0,0],
                               [0,1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,1,0],
                               [0,0,1],
                               [0,0,1],
                               [0,1,0],
                               [1,0,0],
                               [-1,0,0],
                               [0,0,1],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0],
                               [0,-1,0]])

vibe_raw_offsets = np.array([[0,0,0],
                               [0,-1,0],
                               [0,-1,0],
                               [-1,0,0],
                               [0,0,-1],
                               [0,-1,0],
                               [0,-1,0],
                               [0,0,-1],
                               [0,-1,0],
                               [0,1,0],
                               [0,1,0],
                               [0,1,0],
                               [0,1,0],
                               [0,1,0],
                               [0,1,0],
                               [0,1,0],
                               [0,1,0],
                               [0,-1,0]])

mocap_raw_offsets = np.array([[0, 0, 0],
                             [0, 1, 0],
                             [0, 1, 0],
                             [0, 1, 0],
                             [0, 1, 0],
                             [1, 0, 0],
                             [0, 1, 0],
                             [0, 1, 0],
                             [0, 1, 0],
                             [-1, 0, 0],
                             [-1, 0, 0],
                             [-1, 0, 0],
                             [0, -1, 0],
                             [0, -1, 0],
                             [0, -1, 0],
                             [0, -1, 0],
                             [0, -1, 0],
                             [0, -1, 0],
                             [0, -1, 0],
                             [0, -1, 0]])

# Define a kinematic tree for the skeletal struture
humanact12_kinematic_chain = [[0, 1, 4, 7, 10], [0, 2, 5, 8, 11], [0, 3, 6, 9, 12, 15], [9, 13, 16, 18, 20, 22], [9, 14, 17, 19, 21, 23]]

mocap_kinematic_chain = [[0, 1, 2, 3], [0, 12, 13, 14, 15], [0, 16, 17, 18, 19], [1, 4, 5, 6, 7], [1, 8, 9, 10, 11]]

vibe_kinematic_chain = [[0, 12, 13, 14, 15], [0, 9, 10, 11, 16], [0, 1, 8, 17], [1, 5, 6, 7], [1, 2, 3, 4]]

mocap_action_enumerator = {
    0: "Walk",
    1: "Wash",
    2: "Run",
    3: "Jump",
    4: "Animal Behavior",
    5: "Dance",
    6: "Step",
    7: "Climb"
}

humanact12_coarse_action_enumerator = {
    1: "warm_up",
    2: "walk",
    3: "run",
    4: "jump",
    5: "drink",
    6: "lift_dumbbell",
    7: "sit",
    8: "eat",
    9: "turn steering wheel",
    10: "phone",
    11: "boxing",
    12: "throw",
}

humanact12_fine_action_enumerator = {
    101: "warm_up_wristankle",
    102: "warm_up_pectoral",
    103: "warm_up_eblowback",
    104: "warm_up_bodylean_right_arm",
    105: "warm_up_bodylean_left_arm",
    106: "warm_up_bow_right",
    107: "warm_up_bow_left",
    201: "walk",
    301: "run",
    401: "jump_handsup",
    402: "jump_vertical",
    501: "drink_bottle_righthand",
    502: "drink_bottle_lefthand",
    503: "drink_cup_righthand",
    504: "drink_cup_lefthand",
    505: "drink_both_hands",
    601: "lift_dumbbell with _right hand",
    602: "lift_dumbbell with _left hand",
    603: "Lift dumbells with both hands",
    604: "lift_dumbbell over head",
    605: "lift_dumbells with both hands and bend legs",
    701: "sit",
    801: "eat_finger_right",
    802: "eat_pie or hamburger",
    803: "Eat with left hand",
    901: "Turn steering wheel",
    1001: "Take out phone, call and put phone back",
    1002: "Call with left hand",
    1101: "boxing_left_right",
    1102: "boxing_left_upwards",
    1103: "boxing_right_upwards",
    1104: "boxing_right_left",
    1201: "throw_right_hand",
    1202: "throw_both_hands",
}

humanact12_action_enumerator = {
    0: 'warm_up_wristankle',
    1: 'warm_up_pectoral',
    2: 'warm_up_eblowback',
    3: 'walk',
    4: 'run',
    5: 'jump_handsup',
    6: 'drink_bottle_righthand',
    7: 'drink_bottle_lefthand',
    8: 'lift_dumbles_right',
    9: 'lift_dumbles_left',
    10: 'lift_dumbles_both',
    11: 'lift_dumbles_updown',
    12: 'warm_up_bodylean',
    13: 'warm_up_bow_right',
    14: 'warm_up_bow_left',
    15: 'jump_vertical',
    16: 'lift_dumbles_wholeupdown',
    17: 'sit',
    18: 'eat_finger_right',
    19: 'eat_pie',
    20: 'drive',
    21: 'phone_right',
    22: 'box_left_right',
    23: 'box_left',
    24: 'box_right',
    25: 'throw_right',
    26: 'drink_cup_righthand',
    27: 'drink_cup_righthand',
    28: 'eat_finger_left',
    29: 'phone_left',
    30: 'throw_both',
                      }

ntu_action_enumerator = {
    1: "drink water",
    2: "eat meal or snack",
    3: "brushing teeth",
    4: "brushing hair",
    5: "drop",
    6: "pickup",
    7: "throw",
    8: "sitting down",
    9: "standing up (from sitting position)",
    10: "clapping",
    11: "reading",
    12: "writing",
    13: "tear up paper",
    14: "wear jacket",
    15: "take off jacket",
    16: "wear a shoe",
    17: "take off a shoe",
    18: "wear on glasses",
    19: "take off glasses",
    20: "put on a hat or cap",
    21: "take off a hat or cap",
    22: "cheer up",
    23: "hand waving",
    24: "kicking something",
    25: "reach into pocket",
    26: "hopping (one foot jumping)",
    27: "jump up",
    28: "make a phone call or answer phone",
    29: "playing with phone or tablet",
    30: "typing on a keyboard",
    31: "pointing to something with finger",
    32: "taking a selfie",
    33: "check time (from watch)",
    34: "rub two hands together",
    35: "nod head or bow",
    36: "shake head",
    37: "wipe face",
    38: "salute",
    39: "put the palms together",
    40: "cross hands in front (say stop)",
    41: "sneeze or cough",
    42: "staggering",
    43: "falling",
    44: "touch head (headache)",
    45: "touch chest (stomachache or heart pain)",
    46: "touch back (backache)",
    47: "touch neck (neckache)",
    48: "nausea or vomiting condition",
    49: "use a fan (with hand or paper) or feeling warm",
    50: "punching or slapping other person",
    51: "kicking other person",
    52: "pushing other person",
    53: "pat on back of other person",
    54: "point finger at the other person",
    55: "hugging other person",
    56: "giving something to other person",
    57: "touch other person's pocket",
    58: "handshaking",
    59: "walking towards each other",
    60: "walking apart from each other",
    61: "put on headphone",
    62: "take off headphone",
    63: "shoot at the basket",
    64: "bounce ball",
    65: "tennis bat swing",
    66: "juggling table tennis balls",
    67: "hush (quite)",
    68: "flick hair",
    69: "thumb up",
    70: "thumb down",
    71: "make ok sign",
    72: "make victory sign",
    73: "staple book",
    74: "counting money",
    75: "cutting nails",
    76: "cutting paper (using scissors)",
    77: "snapping fingers",
    78: "open bottle",
    79: "sniff (smell)",
    80: "squat down",
    81: "toss a coin",
    82: "fold paper",
    83: "ball up paper",
    84: "play magic cube",
    85: "apply cream on face",
    86: "apply cream on hand back",
    87: "put on bag",
    88: "take off bag",
    89: "put something into a bag",
    90: "take something out of a bag",
    91: "open a box",
    92: "move heavy objects",
    93: "shake fist",
    94: "throw up cap or hat",
    95: "hands up (both hands)",
    96: "cross arms",
    97: "arm circles",
    98: "arm swings",
    99: "running on the spot",
    100: "butt kicks (kick backward)",
    101: "cross toe touch",
    102: "side kick",
    103: "yawn",
    104: "stretch oneself",
    105: "blow nose",
    106: "hit other person with something",
    107: "wield knife towards other person",
    108: "knock over other person (hit with body)",
    109: "grab other person’s stuff",
    110: "shoot at other person with a gun",
    111: "step on foot",
    112: "high-five",
    113: "cheers and drink",
    114: "carry something with other person",
    115: "take a photo of other person",
    116: "follow other person",
    117: "whisper in other person’s ear",
    118: "exchange things with other person",
    119: "support somebody with hand",
    120: "finger-guessing game (playing rock-paper-scissors)",
     }



In [73]:
data_path='/home/youngjoon/Desktop/Dataset/mocap_3djoints/'

In [74]:
file_list=[]
file_list=os.listdir(data_path)

In [75]:
file_list[0]

'P069_61.npy'

In [76]:
clip = pd.read_csv('/home/youngjoon/Desktop/Dataset/pose_clip.csv', index_col=False).dropna(how='all').dropna(axis=1, how='all')

In [77]:
lengths = []
data = []
labels = []

In [78]:
save_root='/home/youngjoon/DEV/ntu-skeleton/code/'

In [85]:
# def inverse_kinemetics(joints):
#         # joints (batch_size, joints_num, 3)
#         # lie_params (batch_size, joints_num, 3)
#         lie_params = tensor(joints.shape).fill_(0)
#         # root_matR (batch_size, 3, 3)
#         root_matR = torch.eye(3, dtype=joints.dtype).expand((joints.shape[0], -1, -1)).clone().detach().to(joints.device)
#         for chain in self._kinematic_tree:
#             R = root_matR
#             for j in range(len(chain) - 1):
#                 # (batch, 3)
#                 u = self._raw_translation[chain[j + 1]].expand(joints.shape[0], -1).clone().detach().to(joints.device)
#                 # (batch, 3)
#                 v = joints[:, chain[j+1], :] - joints[:, chain[j], :]
#                 # (batch, 3)
#                 v = v / torch.norm(v, p=2, dim=1)[:, None]
#                 # (batch, 3, 3)
#                 R_local = torch.matmul(R.transpose(1, 2), lie_exp_map(lie_u_v(u, v)))
#                 # print("R_local shape:" + str(R_local.shape))
#                 # print(R_local)
#                 lie_params[:, chain[j + 1], :] = matR_log_map(R_local)
#                 R = torch.matmul(R, R_local)
#         return lie_params

In [95]:
for i in range(clip.shape[0]):
            motion_name = 'P0'+clip.iloc[i]['motion']
            action_type = clip.iloc[i]['action_type']
            npy_path = os.path.join(data_path, motion_name + '.npy')
            # motion_length, joints_num, 3
            pose_raw = np.load(npy_path)
            # rescale the pose
            pose_raw = pose_raw / 20
            off_set=np.tile(pose_raw[0,0],(pose_raw.shape[1],1))
            pose_mat=pose_raw-mocap_raw_offsets
            pose_mat=pose_mat.reshape((-1,20*3))
            pose_mat = pose_mat[:, 3:]
            data.append((pose_mat, action_type))
            if action_type not in labels:
                labels.append(action_type)
            lengths.append(pose_mat.shape[0])
            cumsum = np.cumsum([0] + lengths)
            print(" Total number of frames {}, videos {}, action types {}".format(cumsum[-1],clip.shape[0],
                                                                             len(labels)))
            label_enc = dict(zip(labels, np.arange(len(labels))))
            label_enc_rev = dict(zip(np.arange(len(labels)),labels))
            with codecs.open(os.path.join(save_root, "label_enc_rev_mocap.txt"), 'w', 'utf-8') as f:
                for item in label_enc_rev.items():
                    f.write(str(item) + "\n")

 Total number of frames 1020500, videos 1088, action types 8
 Total number of frames 1020530, videos 1088, action types 8
 Total number of frames 1020574, videos 1088, action types 8
 Total number of frames 1020611, videos 1088, action types 8
 Total number of frames 1021068, videos 1088, action types 8
 Total number of frames 1021541, videos 1088, action types 8
 Total number of frames 1021601, videos 1088, action types 8
 Total number of frames 1021651, videos 1088, action types 8
 Total number of frames 1021683, videos 1088, action types 8
 Total number of frames 1021716, videos 1088, action types 8
 Total number of frames 1021758, videos 1088, action types 8
 Total number of frames 1021803, videos 1088, action types 8
 Total number of frames 1021855, videos 1088, action types 8
 Total number of frames 1021897, videos 1088, action types 8
 Total number of frames 1021935, videos 1088, action types 8
 Total number of frames 1021972, videos 1088, action types 8
 Total number of frames 

 Total number of frames 1098080, videos 1088, action types 8
 Total number of frames 1098350, videos 1088, action types 8
 Total number of frames 1098364, videos 1088, action types 8
 Total number of frames 1098623, videos 1088, action types 8
 Total number of frames 1099021, videos 1088, action types 8
 Total number of frames 1099562, videos 1088, action types 8
 Total number of frames 1099880, videos 1088, action types 8
 Total number of frames 1100067, videos 1088, action types 8
 Total number of frames 1100290, videos 1088, action types 8
 Total number of frames 1100586, videos 1088, action types 8
 Total number of frames 1100863, videos 1088, action types 8
 Total number of frames 1101172, videos 1088, action types 8
 Total number of frames 1101362, videos 1088, action types 8
 Total number of frames 1101514, videos 1088, action types 8
 Total number of frames 1101851, videos 1088, action types 8
 Total number of frames 1102040, videos 1088, action types 8
 Total number of frames 

 Total number of frames 1144012, videos 1088, action types 8
 Total number of frames 1144048, videos 1088, action types 8
 Total number of frames 1144083, videos 1088, action types 8
 Total number of frames 1144117, videos 1088, action types 8
 Total number of frames 1144140, videos 1088, action types 8
 Total number of frames 1144165, videos 1088, action types 8
 Total number of frames 1144190, videos 1088, action types 8
 Total number of frames 1144212, videos 1088, action types 8
 Total number of frames 1144233, videos 1088, action types 8
 Total number of frames 1144257, videos 1088, action types 8
 Total number of frames 1144309, videos 1088, action types 8
 Total number of frames 1144342, videos 1088, action types 8
 Total number of frames 1144363, videos 1088, action types 8
 Total number of frames 1144384, videos 1088, action types 8
 Total number of frames 1144408, videos 1088, action types 8
 Total number of frames 1144430, videos 1088, action types 8
 Total number of frames 

 Total number of frames 1206597, videos 1088, action types 8
 Total number of frames 1206650, videos 1088, action types 8
 Total number of frames 1206701, videos 1088, action types 8
 Total number of frames 1206754, videos 1088, action types 8
 Total number of frames 1206801, videos 1088, action types 8
 Total number of frames 1206852, videos 1088, action types 8
 Total number of frames 1206905, videos 1088, action types 8
 Total number of frames 1206957, videos 1088, action types 8
 Total number of frames 1207495, videos 1088, action types 8
 Total number of frames 1207550, videos 1088, action types 8
 Total number of frames 1207604, videos 1088, action types 8
 Total number of frames 1207659, videos 1088, action types 8
 Total number of frames 1207712, videos 1088, action types 8
 Total number of frames 1207811, videos 1088, action types 8
 Total number of frames 1207905, videos 1088, action types 8
 Total number of frames 1208000, videos 1088, action types 8
 Total number of frames 

In [110]:
pose_mat

array([[-0.58497706,  0.17957216, -0.03270091, ..., -0.59499015,
         1.00394535,  0.00477642],
       [-0.61383046,  0.19431776, -0.03136107, ..., -0.59195932,
         1.00584725,  0.00764862],
       [-0.64276651,  0.18334706, -0.02970788, ..., -0.59469605,
         1.00620419,  0.00820758],
       ...,
       [-0.4214869 ,  0.16851984,  0.04628281, ...,  0.17251565,
         1.08415836, -0.09196775],
       [-0.47396789,  0.18693763,  0.0579913 , ...,  0.1875504 ,
         1.22361762, -0.09623088],
       [-0.51130275,  0.19751101,  0.06818427, ...,  0.11458594,
         1.40419701, -0.09172163]])

In [109]:
label = label_enc[label]

NameError: name 'label' is not defined

In [104]:
pose_mat[0].shape

(57,)

In [90]:
labels

['Walk', 'Wash', 'Run', 'Jump', 'Animal Behavior', 'Dance', 'Step', 'Climb']

In [92]:
label_enc_rev.get(enc_label)

NameError: name 'enc_label' is not defined

In [42]:
print(pose_mat)
print("-"*40)
print(pose_mat.shape)

[[-0.495655    0.90624    -0.0211039  ... -0.59499015  1.00394535
   0.00477642]
 [-0.527555    0.91996    -0.02068255 ... -0.59195932  1.00584725
   0.00764862]
 [-0.56265     0.907115   -0.0179704  ... -0.59469605  1.00620419
   0.00820758]
 ...
 [-0.3138085   0.90282     0.0206775  ...  0.17251565  1.08415836
  -0.09196775]
 [-0.3706305   0.91993     0.02977285 ...  0.1875504   1.22361762
  -0.09623088]
 [-0.4180025   0.92716     0.0379893  ...  0.11458594  1.40419701
  -0.09172163]]
----------------------------------------
(772, 60)


In [80]:
label_enc_rev.get(enc_label)

NameError: name 'enc_label' is not defined